![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, I'll be creating a data pipeline to analyze Walmart's supply and demand around public holidays, focusing on grocery sales and complementary data. Walmart's e-commerce business reached $80 billion by the end of 2022, with public holidays like the Super Bowl, Labour Day, and Christmas significantly impacting sales.

I'll be working with two main data sources:  
1. **grocery_sales (PostgreSQL)**: Includes `Store_ID`, `Date`, and `Weekly_Sales`.  
2. **extra_data.parquet**: Contains holiday info (`IsHoliday`), economic indicators (`CPI`, `Unemployment`), weather data (`Temperature`, `Fuel_Price`), and store details (`Dept`, `Size`, `Type`).

### Steps:
1. **Merge and Transform Data**: I'll merge both datasets and clean the data to create `clean_data`, which will include:
   - `"Store_ID"`, `"Month"`, `"Dept"`, `"IsHoliday"`, `"Weekly_Sales"`, `"CPI"`, `"Unemployment"`.
   
2. **Monthly Sales Analysis**: I'll aggregate weekly sales by month to identify trends in `agg_data`.

3. **Data Export**: I'll save `clean_data.csv` and `agg_data.csv` for further analysis.

### Next Steps:
- I'll refine the analysis and explore how public holidays, economic factors, and store characteristics affect sales.
- This cleaned and aggregated data will help forecast and optimize Walmart's operations during holiday periods.

In [1]:
-- Write your SQL query here
SELECT * FROM grocery_sales

index  Store_ID       Date  Dept  Weekly_Sales
0            0         1 2010-02-05     1      24924.50
1            1         1 2010-02-05    26      11737.12
2            2         1 2010-02-05    17      13223.76
3            3         1 2010-02-05    45         37.44
4            4         1 2010-02-05    28       1085.29
...        ...       ...        ...   ...           ...
231517  232414        24 2011-05-06     8      49471.07
231518  232415        24 2011-05-06    50       1210.00
231519  232416        24 2011-05-06    87      25893.32
231520  232417        24 2011-05-06    85       1357.83
231521  232418        24 2011-05-06    35       3648.91

[231522 rows x 5 columns]

In [2]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")

In [10]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    # Fill missing numerical values; for example we could use forward fill
    raw_data.fillna(method='ffill', inplace=True)
    
    # Convert 'Date' column to datetime if it's not already
    raw_data['Date'] = pd.to_datetime(raw_data['Date'])
    
     # Add a new column 'Month'
    raw_data['Month'] = raw_data['Date'].dt.month
    
    # Assuming there is a 'Weekly_Sales' column in your DataFrame
    cleaned_data = raw_data[raw_data['Weekly_Sales'] > 10000]

    # Drop unnecessary columns; specify which ones you want to drop here
    cleaned_data.drop(columns=['Store_ID', 'Dept'], inplace=True) 
    return cleaned_data

In [15]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)
clean_data.head()

index       Date  Weekly_Sales  ...  Type      Size  Month
0      0 2010-02-05      24924.50  ...   3.0  151315.0      2
1      1 2010-02-05      11737.12  ...   3.0  151315.0      2
2      2 2010-02-05      13223.76  ...   3.0  151315.0      2
5      5 2010-02-05      46729.77  ...   3.0  151315.0      2
6      6 2010-02-05      21249.31  ...   3.0  151315.0      2

[5 rows x 16 columns]

In [14]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    return (clean_data[['Month', 'Weekly_Sales']]
            .groupby('Month')
            .agg({'Weekly_Sales': 'mean'})
            .reset_index()
            .round(2))
    pass

In [19]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data=avg_weekly_sales_per_month(clean_data)

In [17]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index=False)
    agg_data.to_csv(agg_data_file_path, index=False)
    pass

In [20]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths
# Assuming cleaned_data and aggregated_data are your DataFrames
# Define file paths
cleaned_data_file_path = 'clean_data.csv'
aggregated_data_file_path = 'agg_data.csv'

# Call the load() function with the respective arguments
load(clean_data, cleaned_data_file_path, agg_data, aggregated_data_file_path)


In [22]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    """
    Validates whether the file at file_path exists and is not empty.
    
    Parameters:
    - file_path: str - The path to the file to validate.
    
   :
    - bool: True if the file exists and is not empty, False otherwise.
    """
    
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"File does not exist: {file_path}")
        return False
    
    # Check if the file is not empty
    if os.path.getsize(file_path) == 0:
        print(f"File is empty: {file_path}")
        return False
    
    print(f"File validation successful: {file_path}")
    return True

In [24]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
cleaned_data_file_path = 'clean_data.csv'
aggregated_data_file_path = 'agg_data.csv'

is_cleaned_valid = validation(cleaned_data_file_path)
is_aggregated_valid = validation(aggregated_data_file_path)

print(f"Cleaned Data Validation: {is_cleaned_valid}")
print(f"Aggregated Data Validation: {is_aggregated_valid}")


File validation successful: clean_data.csv
File validation successful: agg_data.csv
Cleaned Data Validation: True
Aggregated Data Validation: True
